In [2]:
from PIL import Image
import requests
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

dataset = load_dataset('eduvedras/Desc_Questions',split='test',trust_remote_code=True)
#dataset[0]['Questions']

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/eduvedras/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#import huggingface_hub

#huggingface_hub.login('',True,False,False)
filetag = "mistral-finetuned"
model_id = "eduvedras/Mistral-finetuned"
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.float16)#.to(device)


Loading checkpoint shards: 100%|██████████| 3/3 [00:09<00:00,  3.07s/it]


In [6]:
from tqdm import tqdm
predictions = []
references = []
for i in tqdm(range(len(dataset))):
#i=14
    inputs = []
    questions = dataset[i]["Questions"][2:-2].split("\', \'")
    inputs.append({"role": "user", "content": "Consider the following data chart \"" +dataset[i]["Description"] + "\", generate a true or false sentence based on this data chart. In your answer only generate the sentence."})
    model_inputs = tokenizer.apply_chat_template(inputs, return_tensors="pt")#.to(device)

    generated_ids = model.generate(model_inputs, max_new_tokens=200, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    result = decoded[0].split("[/INST]")[1] 
    result = result.split("</s>")[0]
    predictions.append(result)
    references.append(dataset[i]["Questions"])
    print(inputs)
    print("---------")
    print(result)


  0%|          | 0/15 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [1]:
import random
predictions_final = []

for el in predictions:
    preds = el.split("\"")
    print(preds)
    if "1." and "5." in preds:
        rand_int = random.randint(1,5)
        preds = preds.split(f"{rand_int}. ")[1]
        preds = preds.split(".")[0]
        preds = preds + "."
        predictions_final.append(preds)
    if len(preds) == 1:
        preds = el.split("(")[0]
        predictions_final.append(preds)
    else:
        predictions_final.append(preds[1])
        
predictions_final

NameError: name 'predictions' is not defined

In [8]:
import evaluate

file = open(f"./resultados-ger-perguntas/{filetag}.txt", "a")

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)
file.write(f"BLEU: {results}\n")

meteor = evaluate.load("meteor")
results = meteor.compute(predictions=predictions, references=references)
print(results)
file.write(f"METEOR: {results}\n")

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=predictions, references=references)
print(results)
file.write(f"ROUGE: {results}\n")
file.close()

{'bleu': 0.025748829412799375, 'precisions': [0.3368146214099217, 0.10386151797603196, 0.0625, 0.03744798890429958], 'brevity_penalty': 0.27068797323198596, 'length_ratio': 0.4335031126202603, 'translation_length': 766, 'reference_length': 1767}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/eduvedras/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/eduvedras/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/eduvedras/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.17508784528307364}
{'rouge1': 0.21018705007664884, 'rouge2': 0.060525853011167495, 'rougeL': 0.15890013636649433, 'rougeLsum': 0.1581472465104503}


In [9]:
import pandas as pd

new_df = pd.DataFrame(columns=['Image','Prediction','References'])

i=0
while i < len(dataset):
    if ((new_df['Image'] == dataset[i]['Chart_name']) & (new_df['Prediction'] == predictions[i])).any():
        i += 1
        continue
    else:
        new_df.loc[len(new_df)] = {'Image': dataset[i]['Chart_name'], 'Prediction': predictions[i], 'References': references[i]}
        i += 1
    
new_df.to_csv(f'./resultados-ger-perguntas/{filetag}.csv', index=False)